<a href="https://colab.research.google.com/github/masifkhan3/CSGC/blob/main/CSHB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def calculate_chlorine_hcl_hydrogen_costs(caustic_soda_prod, sodium_hypo_prod, liquid_chlorine_prod, stearic_batches, hcl_hydrogen_usage, stearic_hydrogen_usage, power_rate, steam_cost, demin_water_cost, chemical_cost, caustic_soda_sale_price, sodium_hypo_sale_price, hcl_sale_price):
    # Constants for calculations
    chlorine_factor = 0.889  # Chlorine production as a fraction of Caustic Soda production
    hypo_chlorine_usage = 0.22  # 22% of Hypo production uses Chlorine
    chlorine_neutralization = 0.017  # 1.7% of Chlorine is neutralized
    hcl_chlorine_usage = 0.32  # 32% of Chlorine is used to produce HCl
    in_house_hcl = 0.05  # 5% of Caustic Soda used in-house for HCl production
    power_rate_per_ton = 2400.0  # Power consumption per ton of Caustic Soda in KWH
    hydrogen_production_percentage = 0.026  # Hydrogen production as 2.6% of Caustic Soda production

    # ---- Chlorine and HCl calculations ----
    chlorine_production = caustic_soda_prod * chlorine_factor  # Chlorine production based on Caustic Soda
    chlorine_used_in_hypo = sodium_hypo_prod * hypo_chlorine_usage  # Chlorine used in Hypo production
    chlorine_neutralized = chlorine_production * chlorine_neutralization  # Chlorine neutralized
    net_chlorine_available = chlorine_production - chlorine_used_in_hypo - chlorine_neutralized - liquid_chlorine_prod  # Chlorine left for HCl production

    hcl_prod = net_chlorine_available / hcl_chlorine_usage  # HCl production
    hcl_in_house = caustic_soda_prod * in_house_hcl  # HCl used in-house
    net_hcl_for_sale = hcl_prod - hcl_in_house  # HCl available for sale

    # ---- Hydrogen calculations ----
    hydrogen_prod_mt = caustic_soda_prod * hydrogen_production_percentage  # Hydrogen production in metric tons (MT)
    hydrogen_prod_nm3 = hydrogen_prod_mt * 34819 / 3.12  # Convert to normal cubic meters (NM3) based on 3.12 MT

    hydrogen_used_in_hcl = hcl_hydrogen_usage  # Hydrogen used in HCl production
    hydrogen_used_in_stearic = stearic_hydrogen_usage  # Hydrogen used in Stearic Acid batches
    total_hydrogen_usage = hydrogen_used_in_hcl + hydrogen_used_in_stearic  # Total hydrogen used
    balance_hydrogen_nm3 = hydrogen_prod_nm3 - total_hydrogen_usage  # Remaining hydrogen after usage
    balance_waste_percentage = (balance_hydrogen_nm3 / hydrogen_prod_nm3) * 100 if hydrogen_prod_nm3 > 0 else 0  # Percentage of hydrogen wasted

    # ---- Power consumption calculations ----
    total_power_used = power_rate_per_ton * caustic_soda_prod  # Total power used based on production
    power_per_ton_caustic_soda = total_power_used / caustic_soda_prod if caustic_soda_prod > 0 else 0  # Power used per ton of caustic soda

    # ---- Cost Calculations ----
    power_factor = 2380  # Power factor in KWH per ton
    steam_factor = 1.37  # Steam factor in tons per ton of caustic soda
    dw_factor = 10.50  # Demin Water factor in m3 per ton
    caustic_soda_percentage = 2.5 / 100  # Caustic Soda percentage
    caustic_soda_cost = 2.96  # Caustic soda cost

    # Calculating the individual costs based on user input
    power_cost = round(power_factor * (power_rate / 1000), 2)  # Power cost per ton
    steam_cost_total = round(steam_factor * (steam_cost / 1000), 2)  # Steam cost per ton
    dw_cost = round(dw_factor * (demin_water_cost / 1000), 2)  # Demin Water cost per ton
    chemical_cost = round(chemical_cost, 2)  # Other Chemicals Cost per ton

    # Total cost per ton of production
    cost_of_production = round(power_cost + steam_cost_total + dw_cost + chemical_cost, 2)

    # Caustic Soda Usage Cost as 2.5% of Cost of Production
    caustic_soda_usage_cost = round(cost_of_production * caustic_soda_percentage, 2)  # Updated calculation

    # Total cost per ton including Caustic Soda Usage Cost
    total_cost_per_ton = round(cost_of_production + caustic_soda_usage_cost, 2)

    # ---- Sales Calculations ----
    total_sales = round((caustic_soda_prod * caustic_soda_sale_price) + (sodium_hypo_prod * sodium_hypo_sale_price) + (net_hcl_for_sale * hcl_sale_price), 2)

    # Sodium Hypo Cost calculation as 22% of Total Cost per ton
    sodium_hypo_cost = round(0.22 * total_cost_per_ton, 2)  # 22% of Total Cost per ton

    # ---- RMC Calculation ----
    rmc_caustic_soda = round(total_cost_per_ton * caustic_soda_prod, 2)
    rmc_sodium_hypo = sodium_hypo_cost * sodium_hypo_prod  # Multiply by the production quantity
    total_rmc = round(rmc_caustic_soda + rmc_sodium_hypo, 2)

    # ---- GC Calculation ----
    total_gc = round(total_sales - total_rmc, 2)
    gc_per_kg = round(total_gc / caustic_soda_prod, 2) if caustic_soda_prod > 0 else 0
    gc_percentage = round((total_gc / total_sales) * 100, 2) if total_sales > 0 else 0  # GC Percentage

    # ---- Results Storage ----
    results = {
        "Production": {
            "Chlorine Production (tons/day)": round(chlorine_production, 2),
            "Chlorine Used in Hypo Production (tons)": round(chlorine_used_in_hypo, 2),
            "Chlorine Neutralized (tons)": round(chlorine_neutralized, 2),
            "Net Chlorine Available for HCl (tons)": round(net_chlorine_available, 2),
            "Total HCl Production (tons)": round(hcl_prod, 2),
            "HCl Used In-House (tons)": round(hcl_in_house, 2),
            "Net HCl Available for Sale (tons)": round(net_hcl_for_sale, 2),
            "Hydrogen Production (MT)": round(hydrogen_prod_mt, 2),
            "Hydrogen Production (NM³)": round(hydrogen_prod_nm3, 2),
            "Balance Hydrogen NM³": round(balance_hydrogen_nm3, 2),
            "Balance Hydrogen Waste (%)": round(balance_waste_percentage, 2),
            "Total Power Used (KWH)": round(total_power_used, 2),
            "Power Used per ton Caustic Soda (KWH)": round(power_per_ton_caustic_soda, 2),
        },
        "Power": {
            "Power Cost (Rs/ton)": power_cost,
            "Steam Cost (Rs/ton)": steam_cost_total,
            "Demin Water Cost (Rs/ton)": dw_cost,
            "Other Chemical Costs (Rs/ton)": chemical_cost,
            "Total Cost of Production (Rs/ton)": cost_of_production,
            "Caustic Soda Usage Cost (Rs/ton)": caustic_soda_usage_cost,
            "Total Cost per ton (Rs)": total_cost_per_ton,
            "Sodium Hypo Cost (Rs)": sodium_hypo_cost,  # Add Sodium Hypo Cost to the Power section
        },
        "Cost": {
            "Sales Total (Rs)": total_sales,
        },
        "RMC": {
            "RMC Caustic Soda (Rs)": rmc_caustic_soda,
            "RMC Sodium Hypochlorite (Rs)": rmc_sodium_hypo,
            "Total RMC (Rs)": total_rmc,
        },
        "GC": {
            "Total GC (Rs)": total_gc,
            "GC per kg (Rs)": gc_per_kg,
            "GC Percentage (%)": gc_percentage,
        }
    }

    return results

# Input data from the user
caustic_soda_prod = float(input("Enter Caustic Soda production in tons (TPD): "))
sodium_hypo_prod = float(input("Enter Sodium Hypochlorite production in tons: "))
stearic_batches = float(input("Enter number of Stearic Acid Batches: "))
liquid_chlorine_prod = float(input("Enter Liquid Chlorine production in tons: "))

# Asking for cost-related inputs from the user
power_rate = float(input("Enter Power Rate (per unit in Rs): "))
steam_cost = float(input("Enter Steam Cost (per ton in Rs): "))
demin_water_cost = float(input("Enter Demin Water Cost (per m3 in Rs): "))
chemical_cost = float(input("Enter Other Chemical Costs (per ton in Rs): "))

# Asking for sales prices
caustic_soda_sale_price = float(input("Enter Sale Price of Caustic Soda (per ton in Rs): "))
sodium_hypo_sale_price = float(input("Enter Sale Price of Sodium Hypochlorite (per ton in Rs): "))
hcl_sale_price = float(input("Enter Sale Price of HCl (per ton in Rs): "))

# Perform calculations
results = calculate_chlorine_hcl_hydrogen_costs(
    caustic_soda_prod,
    sodium_hypo_prod,
    liquid_chlorine_prod,
    stearic_batches,
    hcl_hydrogen_usage=17228.0,
    stearic_hydrogen_usage=5400.0,
    power_rate=power_rate,
    steam_cost=steam_cost,
    demin_water_cost=demin_water_cost,
    chemical_cost=chemical_cost,
    caustic_soda_sale_price=caustic_soda_sale_price,
    sodium_hypo_sale_price=sodium_hypo_sale_price,
    hcl_sale_price=hcl_sale_price
)

# ---- Convert results to DataFrame for tabulation ----
production_df = pd.DataFrame.from_dict(results["Production"], orient='index', columns=['Value'])
power_df = pd.DataFrame.from_dict(results["Power"], orient='index', columns=['Value'])
cost_df = pd.DataFrame.from_dict(results["Cost"], orient='index', columns=['Value'])
rmc_df = pd.DataFrame.from_dict(results["RMC"], orient='index', columns=['Value'])
gc_df = pd.DataFrame.from_dict(results["GC"], orient='index', columns=['Value'])

# Displaying results
print("\n---- Production ----")
print(production_df)

print("\n---- Power ----")
print(power_df)

print("\n---- Cost ----")
print(cost_df)

print("\n---- RMC ----")
print(rmc_df)

print("\n---- GC ----")
print(gc_df)
